In [12]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder, StandardScaler
import os

# Đọc dữ liệu
df = pd.read_csv('dataProcessed.csv')
print(df.head())

     location rooms  sqm_m2  price_normalized  trieu_met_vuong  \
0  Binh Chanh  1 PN      38             760.0            20.00   
1  Binh Chanh  2 PN      72            2780.0            38.61   
2  Binh Chanh  2 PN      72            2750.0            38.19   
3  Binh Chanh  2 PN      72            2249.0            31.24   
4  Binh Chanh  2 PN      70            2800.0            40.00   

   rooms_normalized  
0                 1  
1                 2  
2                 2  
3                 2  
4                 2  


In [14]:
# Mã hóa cột 'location' thành số nguyên
label_encoder_location = LabelEncoder()
df['location_encoded'] = label_encoder_location.fit_transform(df['location'])

# Chuẩn hóa dữ liệu (scaling) để đưa các đặc trưng về cùng một khoảng giá trị
scaler = StandardScaler()
df[['location_encoded', 'rooms_normalized', 'sqm_m2']] = scaler.fit_transform(df[['location_encoded', 'rooms_normalized', 'sqm_m2']])

# Hiển thị DataFrame mới sau khi mã hóa và chuẩn hóa
print(df.head())

     location rooms    sqm_m2  price_normalized  trieu_met_vuong  \
0  Binh Chanh  1 PN -1.246975             760.0            20.00   
1  Binh Chanh  2 PN -0.009419            2780.0            38.61   
2  Binh Chanh  2 PN -0.009419            2750.0            38.19   
3  Binh Chanh  2 PN -0.009419            2249.0            31.24   
4  Binh Chanh  2 PN -0.082217            2800.0            40.00   

   rooms_normalized  location_encoded  
0         -1.496385         -1.738916  
1         -0.121108         -1.738916  
2         -0.121108         -1.738916  
3         -0.121108         -1.738916  
4         -0.121108         -1.738916  


In [17]:
# Đặt biến môi trường để giảm số luồng xử lý
os.environ['OMP_NUM_THREADS'] = '8'

# Khởi tạo mô hình KMeans và xác định số cụm (clusters)
k = 3  # Số cụm mong muốn
kmeans_model = KMeans(n_clusters=k, random_state=42, n_init=10)

# Tách dữ liệu thành features
X = df[['location_encoded', 'rooms_normalized', 'sqm_m2']]

# Huấn luyện mô hình KMeans
kmeans_model.fit(X)

# Gán nhãn (label) cụm vào DataFrame
df['cluster'] = kmeans_model.labels_

# Hiển thị DataFrame mới với nhãn cụm
print(df[['location', 'rooms_normalized', 'sqm_m2', 'cluster']].head())

C:\Users\ASUS\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=8.
  warnings.warn(


     location  rooms_normalized    sqm_m2  cluster
0  Binh Chanh         -1.496385 -1.246975        2
1  Binh Chanh         -0.121108 -0.009419        2
2  Binh Chanh         -0.121108 -0.009419        2
3  Binh Chanh         -0.121108 -0.009419        2
4  Binh Chanh         -0.121108 -0.082217        2


In [18]:
# Dự đoán cụm của một căn hộ mới
new_data = pd.DataFrame({
    'location': ['Binh Chanh'],
    'rooms_normalized': [2],
    'sqm_m2': [80]
})

# Sử dụng label encoding cho vị trí của căn hộ mới
new_data['location_encoded'] = label_encoder_location.transform(new_data['location'])

# Chuẩn hóa dữ liệu mới
new_data[['location_encoded', 'rooms_normalized', 'sqm_m2']] = scaler.transform(new_data[['location_encoded', 'rooms_normalized', 'sqm_m2']])

# Dự đoán cụm của căn hộ mới
predicted_cluster = kmeans_model.predict(new_data[['location_encoded', 'rooms_normalized', 'sqm_m2']])
print("Predicted cluster:", predicted_cluster)

Predicted cluster: [2]
